In [3]:
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta
import re
import getpass
import io
import os, json
from py2neo import Graph as GRAPHS


ImportError: cannot import name 'Graph' from 'neographviz' (c:\Users\VBAPHCColleD\AppData\Local\Programs\Python\Python39\lib\site-packages\neographviz\__init__.py)

In [2]:
graph = GRAPHS("bolt://localhost:7687", user="neo4j", password="temp",name="sessions")


In [4]:
azure_log = pd.read_csv (r'C:\Users\user\Downloads\azure_logs.csv',skip_blank_lines=True) 
azure_log.dropna(how="all", inplace=True)
print(azure_log.columns)

Index(['timestamp', 'azure.flow.destination.bytes',
       'azure.flow.destination.ip', 'azure.flow.destination.port',
       'azure.flow.firewall.action', 'azure.flow.source.ip',
       'client.as.organization_name', 'client.geo.continent_name',
       'client.geo.country_name', 'client.ip', 'client.packets',
       'destination.bytes', 'destination.ip', 'event.pipeline.host.os.kernel',
       'event.pipeline.host.os.name', 'event.pipeline.host.os.family',
       'event.pipeline.host.os.platform', 'client.as.ip', 'client.bytes'],
      dtype='object')


In [13]:
try:
    graph.run("Match () Return 1 Limit 1")
    print('ok')
except Exception:
    print('not ok')
tx = graph.begin()
for index, row in azure_log.iterrows():
    tx.evaluate('''
    MERGE (s:host {IP: $SourceIP})
    MERGE (d:host{IP: $DestinationIP})
    CREATE (s)-[i:session {Name: $SourceIP + $DestinationIP + $DestinationPort}]->(d)
    SET i.Bytes = toInteger($bytes),i.DestinationPort=$DestinationPort,i.Time=$time,i.Packets=$packets,i.country=$country
    ''', parameters = {'SourceIP': row['azure.flow.source.ip'], 'DestinationIP': row['azure.flow.destination.ip'],'DestinationPort': row['azure.flow.destination.port'],'packets': row['client.packets'],'bytes': row['destination.bytes'],'time': row['timestamp'],'country': row['client.geo.country_name']})
graph.commit(tx)  

ok


In [5]:
try:
    graph.run("Match () Return 1 Limit 1")
    print('ok')
except Exception:
    print('not ok')
my_node = graph.run("MATCH (n) DETACH DELETE n")

ok
